In [30]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter


# 1. Load Document

In [36]:
txt_doc = "docs/Constitucion_Chilena_1980_s_notas.txt"
with open (txt_doc) as f:
    contents = f.read()

In [31]:
loader = PyPDFLoader("docs/Texto-aprobado-Consejo-Constitucional_06.10.23.pdf")
pages = loader.load()

In [32]:
len(pages)

107

In [33]:
len(pages[0].page_content)

2912

In [34]:
pages[0].page_content[0:2000]

' \n1 \n  \nTEXTO APROBADO POR EL CONSEJO CONSTITUCIONAL  \n \n \nCAPÍTULO I  \n \nFUNDAMENTOS DEL ORDEN CONSTITUCIONAL  \n \nArtículo 1  \n \n1. La dignidad humana es inviolable y la base del derecho y la justicia. Las personas nacen \nlibres e iguales en dignidad y derechos. Su respeto y garantía es el primer deber de la \ncomunidad política y de su forma jurídica de organización.  \n \n2. La familia es el núcleo fundamental de la sociedad. Es deber del Estado y la sociedad dar \nprotección a las familias y propender a su for talecimiento.  \n \n3. Las agrupaciones que libremente surjan entre las personas gozarán de la adecuada \nautonomía para cumplir sus fines específicos que no sean contrarios a la Constitución. El \nEstado respetará los efectos de este reconocimiento . \n \nArtículo 2  \n \n1. El Estado de Chile es social y democrático de derecho, que reconoce  derechos y libertades \nfundamentales, deberes constitucionales, y promueve el desarrollo progresivo de los \nderechos s

In [8]:
pages[0].metadata

{'source': 'docs/Texto-aprobado-Consejo-Constitucional_06.10.23.pdf',
 'page': 0}

# 2. Split Document (1500, 300)

In [41]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", "\. ", " ", ""],
    chunk_size=2000,
    chunk_overlap=1000,
    length_function=len
)

In [42]:
docs = text_splitter.split_documents(pages)

In [43]:
len(docs)

288

In [46]:
print(docs[151].page_content)

públicos y entidades privadas a cargo de la infraestructura crítica del país, así como los 
criterios específicos para la  identificación de la misma.  
 
2. El Presidente de la República, a través de un decreto supremo, designará a un oficial 
general de las Fuerzas Armadas que tendrá el mando de las Fuerzas Armadas y de las Fuerzas  
de Orden y Seguridad Pública dispuestas para l a protección de la infraestructura crítica en 
las áreas especificadas en dicho acto. Los jefes designados para el mando de las fuerzas 
tendrán la responsabilidad del resguardo del orden público en dichas áreas, de acuerdo con 
las instrucciones que establez ca el ministerio a cargo de la Seguridad Pública en el decreto 
supremo.  
 
3. El ejercicio de esta atribución no implicará la suspensión, restricción o limitación de los 
derechos y garantías consagrados en esta Constitución o en tratados internacionales sobr e 
derechos humanos ratificados por Chile y que se encuentren vigentes. Sin perjuicio de lo 


In [45]:
print(docs[150].page_content)

56 
  
 
Disposiciones generales  
 
Artículo 1 14 
 
1. Para efectos de lo dispuesto en el literal r) del artículo 101, la infraestructura crítica 
comprende el conjunto de instalaciones, sistemas físicos o servicios esenciales y de utilidad 
pública, así como aquellos cuya afectación cause un grave daño a la salud o al abastecimiento 
de la población, a la actividad económica esencial, al medioambiente o a la seguridad del 
país. Se entiende por este concepto la infraestructura indispensable para la generación, 
transmisión, transporte, producción, almacenamiento  y distribución de los servicios e 
insumos básicos para la población, tales como energía, gas, agua o telecomunicaciones; la 
relativa a la conexión vial, aérea, terrestre, marítima, portuaria o ferroviaria, y la 
correspondiente a servicios de utilidad públ ica, como los sistemas de asistencia sanitaria o 
de salud. Una ley regulará las obligaciones a las que estarán sometidos los organismos 
públicos y entidades privadas 

# 3. Create Embeddings

In [47]:
from langchain.embeddings import BedrockEmbeddings 
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1")

In [48]:
bedrock_embeddings.embed_query(text="Hola Mundo")[:3]

[0.51953125, 0.44921875, 0.32421875]

In [49]:
from langchain.vectorstores import Chroma

In [50]:
persist_directory = 'streamlit/docs/chroma/'
!rm -rf .streamlit/docs/chroma  # remove old database files if any

In [51]:
%%time
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=bedrock_embeddings,
    persist_directory=persist_directory
)
print(vectordb._collection.count())

884
CPU times: user 1.86 s, sys: 244 ms, total: 2.1 s
Wall time: 1min 34s


In [54]:
%%time
question = "Como se compone el Congreso Nacional?"
responses = vectordb.similarity_search(question,k=8)

CPU times: user 12.3 ms, sys: 3.8 ms, total: 16.1 ms
Wall time: 425 ms


In [58]:
responses[0].metadata

{'page': 35,
 'source': 'docs/Texto-aprobado-Consejo-Constitucional_06.10.23.pdf'}

In [59]:
for resp in responses:
    print (resp.metadata['page'])
    print(resp.page_content[:1000])


35
36 
  
En los casos en que el Estado sea objeto de una demanda o denuncia ante 
organismos internacionales por presuntas vulneraciones de los tratados 
internacionales aprobados por el Congreso Nacional conforme a este artículo, y 
respecto de la cual el Presidente de la República tenga la intención de celebrar o 
acceder a un acuerdo o solución alternativa, deberá ser informado  a ambas 
cámaras, antes de su conclusión, para su conocimiento. Con todo, el Presidente 
de la República no podrá transigir o acordar la realiz ación de acciones o adopción 
de medidas que excedan de las facultades que la Constitución le otorga.  
 
b) Pronunciarse, cuando corresponda, respecto de los estados de excepción 
constitucional, en la forma prescrita por esta Constitución.  
 
 
Funcionamiento de l Congreso Nacional  
 
Artículo 61  
 
1. El Congreso Nacional se instalará e iniciará su período de sesiones en la forma que 
determine su ley institucional.  
 
2. En todo caso, se entenderá siempre co

In [52]:
vectordb.persist()

In [53]:
print(vectordb._collection.count())

884


In [60]:
docs_mmr = vectordb.max_marginal_relevance_search(question,k=8)

In [61]:
for resp in docs_mmr:
    print (resp.metadata['page'])
    print(resp.page_content[:1000])

35
36 
  
En los casos en que el Estado sea objeto de una demanda o denuncia ante 
organismos internacionales por presuntas vulneraciones de los tratados 
internacionales aprobados por el Congreso Nacional conforme a este artículo, y 
respecto de la cual el Presidente de la República tenga la intención de celebrar o 
acceder a un acuerdo o solución alternativa, deberá ser informado  a ambas 
cámaras, antes de su conclusión, para su conocimiento. Con todo, el Presidente 
de la República no podrá transigir o acordar la realiz ación de acciones o adopción 
de medidas que excedan de las facultades que la Constitución le otorga.  
 
b) Pronunciarse, cuando corresponda, respecto de los estados de excepción 
constitucional, en la forma prescrita por esta Constitución.  
 
 
Funcionamiento de l Congreso Nacional  
 
Artículo 61  
 
1. El Congreso Nacional se instalará e iniciará su período de sesiones en la forma que 
determine su ley institucional.  
 
2. En todo caso, se entenderá siempre co

In [1]:
len("c2023-lapplb-kxji7dzouism-1990820049.us-east-1.elb.amazonaws.com")

64